In [1]:
import tensorflow as tf
import pandas as pd 
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_df = pd.read_csv('Dataset/train.csv')
mnist = input_data.read_data_sets("MNIST_data/", one_hot = True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
feat_cols = train_df.drop('label',axis=1)

In [4]:
labels = train_df['label']

In [5]:
test_df = pd.read_csv('Dataset/test.csv')

In [6]:
#INIT WEIGHTS
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

In [7]:
#INIT BIAS
def init_bias(shape):
    init_bias_vals = tf.constant(0.1,shape=shape)
    return tf.Variable(init_bias_vals)


In [8]:
#CONV2D

def conv2d(x,W):
    # x --> input tensor --> [batch,H,W,Channels]
    # W --> [filter H, filter W, Channels IN, Channels OUT]

    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')



In [9]:
#POOLING

def max_pool_2by2(x):
    # x --> input tensor --> [batch,H,W,Channels]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')


In [10]:
# CONVOLUTIONAL LAYER

def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

# NORMAL (FULLY CONNECTED)

def normal_full_layer(input_layer,size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size,size])
    b = init_bias([size])
    return tf.matmul(input_layer,W) + b

In [11]:
# PLACEHOLDERS

x = tf.placeholder(tf.float32,shape = [None,784])

y_true = tf.placeholder(tf.float32,shape=[None,10]) #AS one hot encoded

# LAYERS
# RESHAPE X TO 28 x 28. 1 is grayscale.
x_image = tf.reshape(x, [-1,28,28,1])
# 5 x 5 is patch size. 32 is no. of output channels (features computing)
convo_1 = convolutional_layer(x_image,shape=[5,5,1,32])

convo_1_pooling = max_pool_2by2(convo_1)

convo_2 = convolutional_layer(convo_1_pooling,shape = [5,5,32,64])

convo_2_pooling = max_pool_2by2(convo_2)

# 7 by 7 image times output i.e 64

convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])

# 1024 neurons

full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))

In [12]:
# DROPOUT

hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob=hold_prob)

y_pred = normal_full_layer(full_one_dropout,10)

# LOSS FUNCTION

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

# OPTIMIZER

optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train = optimizer.minimize(cross_entropy)


In [13]:
test_df = pd.read_csv('Dataset/test.csv')

# output = pd.DataFrame({ 'PassengerId' : ids, 'Survived': predictions })
# output.to_csv('titanic-predictions.csv', index = False)
# output.head()

In [14]:
numpyMatrix = test_df.as_matrix()

In [15]:
numpyMatrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [16]:
numpyMatrix_norm = numpyMatrix / 255

In [17]:
# init = tf.global_variables_initializer()
# saver = tf.train.Saver()
# steps = 5000

# with tf.Session() as sess:
#     sess.run(init)

#     for i in range(steps):
#         batch_x, batch_y = mnist.train.next_batch(50)
#         sess.run(train,feed_dict={x:batch_x,y_true:batch_y, hold_prob:0.5})
    
#         if i%100 == 0:
#              print("ON STEP: {}".format(i))
#     saver.save(sess,'model/my_model2.ckpt')
#             print("ACCURACY: ")
#             matches = tf.equal(tf.argmax(y_pred,axis=1), tf.argmax(y_true,1))

#             acc = tf.reduce_mean(tf.cast(matches, tf.float32))

#             print(sess.run(acc,feed_dict={x:mnist.test.images, y_true:mnist.test.labels,hold_prob:1.0}))
#             print('\n')

In [18]:
numpyMatrix_norm.shape

(28000, 784)

In [63]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    saver.restore(sess,'model/my_model2.ckpt')

    for i in range(1):
        
        ass = tf.argmax(y_pred,1)
        acc = ass
#         pred = sess.run(acc,feed_dict={x:numpyMatrix_norm[0:10000],y_true:mnist.test.labels,hold_prob:1.0})
#         preda = sess.run(acc,feed_dict={x:numpyMatrix_norm[10000:20000],y_true:mnist.test.labels,hold_prob:1.0})
#         predb = sess.run(acc,feed_dict={x:numpyMatrix_norm[20000:28000],y_true:mnist.test.labels,hold_prob:1.0})

      
        print("in step {}".format(i))
        

INFO:tensorflow:Restoring parameters from model/my_model2.ckpt
in step 0


In [62]:
preda

array([0, 1, 5, ..., 5, 4, 6], dtype=int64)

In [64]:
predb

array([7, 0, 3, ..., 3, 9, 2], dtype=int64)

In [61]:
pred

array([2, 0, 9, ..., 2, 7, 7], dtype=int64)

In [87]:
predictions_final = []
for i in pred:
    predictions_final.append(i)
for j in preda:
    predictions_final.append(j)
for k in predb:
    predictions_final.append(k)


In [94]:
# predictions_final.shape
ids = np.arange(1,28001,dtype=int)

In [95]:
ids

array([    1,     2,     3, ..., 27998, 27999, 28000])

In [96]:
output = pd.DataFrame({ 'ImageId' : ids, 'Label': predictions_final })
output.to_csv('digit-recognizera.csv', index = False)
output.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
